# Overview
This notebook analyzes the performance of various IK solvers for solution refinement of IKFlow generated IK solutions. Currently implemented IK solvers are TRAC-IK and klamp't

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext wurlitzer
# ^ the wurlitzer extension is used to capture C/C++ output to be displayed in the notebook
# Nov9: Weird behaviour was observed where, if calling klampt's ik function (via 
#     robot.inverse_kinematics_klampt(...)) more than once, there would be no output, and the kernel would hang.
#     This weird behavior ceased when the `%load_ext wurlitzer` call was added. I do not know why. I haven't
#     seen similar behaviour outside of a jupyter notebook.
import os
os.chdir("../")
print("Current working directory:", os.getcwd())

In [ ]:
from time import time
import pickle
import yaml

from ikflow import config
from ikflow.ikflow_solver import IKFlowSolver
from jrl.robots import get_robot
from ikflow.utils import set_seed
from ikflow.ikflow_solver import get_ik_solver

import torch
import numpy as np
import tqdm

set_seed()
np.set_printoptions(edgeitems=10)
np.core.arrayprint._line_width = 180

Verify external matplotlib styles are avialable 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
print("matplotlib_fname:", matplotlib.matplotlib_fname())
print("configdir:       ", matplotlib.get_configdir())
print("available:       ", plt.style.available)

In [ ]:
MODEL_NAME="panda_tpm" # should match an entry in `model_descriptions.yaml`
with open("model_descriptions.yaml", "r") as f:
    MODEL_DESCRIPTIONS = yaml.safe_load(f)

# Save runtime data

In [ ]:
model_weights_filepath = MODEL_DESCRIPTIONS[MODEL_NAME]["model_weights_filepath"]
robot_name = MODEL_DESCRIPTIONS[MODEL_NAME]["robot_name"]
hparams = MODEL_DESCRIPTIONS[MODEL_NAME]

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver(model_weights_filepath, robot_name, hparams)
robot = get_robot(robot_name)

In [ ]:
def quick_set(data, solver_name, prefix, batch_i, mean, std, pct_success):
    data[solver_name][f"{prefix}_runtimes"][batch_i] = mean
    data[solver_name][f"{prefix}_stds"][batch_i] =std
    data[solver_name][f"{prefix}_pct_success"][batch_i] =pct_success

def fn_runtime_stats(fn, n_sols, k=5):
    runtimes = []
    successes = 0
    for _ in range(k):
        t0 = time()
        for sol_i in range(n_sols):
            if fn(sol_i) is not None:
                successes += 1
        runtimes.append(time() - t0)
    return {"mean": np.mean(runtimes), "std": np.std(runtimes), "pct_success": 100*(successes/(k*n_sols))}

In [ ]:
[10 * i for i in range(0, 101, 10)][1:]

In [ ]:
batch_sizes = [10 * i for i in range(0, 101, 10)][1:]
# batch_sizes = [10 * i for i in range(0, 51, 5)][1:] # [50, 100, ..., 450, 500]
# batch_sizes = [5 * i for i in range(0, 21, 5)][1:]
n_batches = len(batch_sizes)
tolerance = 1e-4
n_measures_p_batch = 1

data = {
    "model_name":  MODEL_NAME,
    "batch_sizes": np.array(batch_sizes),
    "tolerance":   tolerance,
    "n_measures_p_batch": n_measures_p_batch,
    "klampt": {
            "random_seed_runtimes":    np.zeros(n_batches),
            "random_seed_stds":        np.zeros(n_batches),
            "random_seed_pct_success": np.zeros(n_batches),
            "ikf_seed_runtimes":       np.zeros(n_batches),
            "ikf_seed_stds":           np.zeros(n_batches),
            "ikf_seed_pct_success":    np.zeros(n_batches),
    },
    "tracik": {
        "random_seed_runtimes":    np.zeros(n_batches),
        "random_seed_stds":        np.zeros(n_batches),
        "random_seed_pct_success": np.zeros(n_batches),
        "ikf_seed_runtimes":       np.zeros(n_batches),
        "ikf_seed_stds":           np.zeros(n_batches),
        "ikf_seed_pct_success":    np.zeros(n_batches),
    },
    "model_runtimes":     np.zeros(n_batches),
    "model_runtime_stds": np.zeros(n_batches)
}

for batch_i, batch_size in tqdm.tqdm(enumerate(batch_sizes)):
    print(f"Solving - batch_size={batch_size} \t({batch_i+1}/{n_batches})")
    
    samples = robot.sample_joint_angles(batch_size)
    target_poses = robot.forward_kinematics_klampt(samples)
    ikflow_sols, ikflow_sol_runtime = ik_solver.solve_n_poses(target_poses, latent_scale=0.75)
    ikflow_sols = ikflow_sols.detach().cpu().numpy()
    data["model_runtimes"][batch_i] = ikflow_sol_runtime
    
    klampt_random_seed_fn=lambda i: robot.inverse_kinematics_klampt(
            target_poses[i], seed=None, positional_tolerance=tolerance, verbosity=0
        )
    klampt_seeded_fn=lambda i: robot.inverse_kinematics_klampt(
            target_poses[i], seed=ikflow_sols[i], positional_tolerance=tolerance, verbosity=0
        )
    tracik_random_seed_fn=lambda i: robot.inverse_kinematics_tracik(
            target_poses[i], seed=None, positional_tolerance=tolerance, verbosity=0
        )
    tracik_seeded_fn=lambda i: robot.inverse_kinematics_tracik(
            target_poses[i], seed=ikflow_sols[i], positional_tolerance=tolerance, verbosity=0
        )
    
    quick_set(data, "klampt", "random_seed", batch_i, **fn_runtime_stats(klampt_random_seed_fn, batch_size))
    quick_set(data, "klampt", "ikf_seed", batch_i,**fn_runtime_stats(klampt_seeded_fn, batch_size))
    quick_set(data, "tracik", "random_seed", batch_i, **fn_runtime_stats(tracik_random_seed_fn, batch_size))
    quick_set(data, "tracik", "ikf_seed", batch_i, **fn_runtime_stats(tracik_seeded_fn, batch_size))
    


In [ ]:
with open(f"notebooks/solution_refinement_runtimes-{robot_name}-{MODEL_NAME}.pkl", "wb") as f:
    pickle.dump(data, f)

# Plotting

Per ieee: "One column width: 3.5 inches, 88.9 millimeters, or 21 picas "

For generating a plot for a paper, use:
```
IEEE_COLUMN_WIDTH = 3.5 # inches
GOLDEN_RATIO = 1.61803398875
figure_height = IEEE_COLUMN_WIDTH / GOLDEN_RATIO
```

In [ ]:
with open(f"notebooks/solution_refinement_runtimes-{robot_name}-{MODEL_NAME}.pkl", "rb") as f:
    data = pickle.load(f)
    batch_sizes = data["batch_sizes"]
    for solver in ["klampt", "tracik"]:
        for entry in ["random_seed_runtimes", "random_seed_stds", "random_seed_pct_success", 
                      "ikf_seed_runtimes", "ikf_seed_stds", "ikf_seed_pct_success"]:
            assert isinstance(data[solver][entry], np.ndarray), \
                f"Entry data['{solver}']['{entry}'] should be a np.ndarray but is {type(data[solver][entry])}"
            assert len(data[solver][entry]) == len(batch_sizes), \
                f"data['{solver}']['{entry}'] has {len(data[solver][entry])} elements, but should have {len(batch_sizes)}"

In [ ]:
# Reset all styles
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
plt.rcdefaults()
matplotlib.rcdefaults()
plt.rcParams.update(plt.rcParamsDefault)

# Plot settings
std_alpha = 0.1
grid_alpha = 0.25

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
from matplotlib.ticker import FormatStrFormatter
figsize = (9, 5)
# Get runtimes from pkl
#     "klampt": {
#             "random_seed_runtimes":    [],
#             "random_seed_stds":        [],
#             "random_seed_pct_success": [],
#             "ikf_seed_runtimes":       [],
#             "ikf_seed_stds":           [],
#             "ikf_seed_pct_success":    [],
#     },
#     "tracik": {
#         "random_seed_runtimes":    [],
#         "random_seed_stds":        [],
#         "random_seed_pct_success": [],
#         "ikf_seed_runtimes":       [],
#         "ikf_seed_stds":           [],
#         "ikf_seed_pct_success":    [],
#     },
#     "model_runtimes":     [],
#     "model_runtime_stds": []

tracik_color = "g"
klampt_color = "r"
ikflow_color = "b"
random_seed_ls = "--"

def make_plots(ax):
    batch_sizes = data["batch_sizes"]
    d = data
    d_kl = data["klampt"]
    d_ti = data["tracik"]
    
    # IKFlow     
    ikf_mean = data["model_runtimes"]
    std = data["model_runtime_stds"]
    ax.plot(batch_sizes, ikf_mean, label="IKFlow")
    ax.fill_between(batch_sizes, ikf_mean - std, ikf_mean + std, alpha=std_alpha, color=ikflow_color)
    
    # Klampt - seeded         
    mean = d_kl["random_seed_runtimes"]
    std = d_kl["random_seed_stds"]
    ax.plot(batch_sizes, mean, label="Klampt - random seed", color=klampt_color)
    ax.fill_between(batch_sizes, mean - std, mean + std, alpha=std_alpha, color=klampt_color)

    # Klampt - random seeds
    mean = d_kl["ikf_seed_runtimes"] + ikf_mean
    std = d_kl["ikf_seed_stds"]
    ax.plot(batch_sizes, mean, label="Klampt - ikflow seed", color=klampt_color, linestyle=random_seed_ls)
    ax.fill_between(batch_sizes, mean - std, mean + std, alpha=std_alpha, color=klampt_color)
        
    # tracik - seeded         
    mean = d_ti["random_seed_runtimes"]
    std = d_ti["random_seed_stds"]
    ax.plot(batch_sizes, mean, label="Trac-ik - random seed", color=tracik_color, linestyle=random_seed_ls)
    ax.fill_between(batch_sizes, mean - std, mean + std, alpha=std_alpha, color=tracik_color)

    # tracik - random seeds
    mean = d_ti["ikf_seed_runtimes"] + ikf_mean
    std = d_ti["ikf_seed_stds"]
    ax.plot(batch_sizes, mean, label="Trac-ik - ikflow seed", color=tracik_color)
    ax.fill_between(batch_sizes, mean - std, mean + std, alpha=std_alpha, color=tracik_color)
    

def make_succ_plot(ax):
    batch_sizes = data["batch_sizes"]
    d_kl = data["klampt"]
    d_ti = data["tracik"]
    
    # Klampt - random seeds
    ax.plot(batch_sizes, d_kl["ikf_seed_pct_success"], label="Klampt, ikflow seed", color=klampt_color)
    ax.plot(batch_sizes, d_kl["random_seed_pct_success"], label="Klampt, random seed", color=klampt_color, linestyle=random_seed_ls)
    ax.plot(batch_sizes, d_ti["ikf_seed_pct_success"], label="Trac-ik, ikflow seed", color=tracik_color)
    ax.plot(batch_sizes, d_ti["random_seed_pct_success"], label="Trac-ik, random seed", color=tracik_color, linestyle=random_seed_ls)
    

    
with plt.style.context("default"):

    fig, axs = plt.subplots(1, 2, figsize=figsize)

    l_plot = axs[0]
    r_plot = axs[1]
    fig.tight_layout(pad=1.08)  # Or equivalently,  "plt.tight_layout()"
    plt.rcParams["font.family"] = "serif"

    # Plot
    make_plots(l_plot)
    make_succ_plot(r_plot)

    # Lables
    fig.suptitle("Number of solutions vs. refinement time", fontweight="bold", fontsize=BIGGER_SIZE)

    l_plot.set_xlabel("Number of solutions")
    l_plot.set_ylabel("Refinement time (sec)")
    l_plot.legend(loc="upper left")
    l_plot.grid(True, alpha=grid_alpha)
    
#     r_plot.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    r_plot.set_xlabel("Number of solutions")
    r_plot.set_ylabel("Success %")
    r_plot.legend(loc="upper left")
    r_plot.grid(True, alpha=grid_alpha)

    plt.subplots_adjust(top=0.9)

#     fig.savefig("results/figures/runtime/runtime.pdf", bbox_inches="tight")
    plt.show()